### MELD

In this notebook we'll merge results from the two techniques used to collect metadata from the phot archive

- **metadata** extraction with exiftool
- **ocr** with tessaract

We'll discuss what we obtained and we'll eventually merge the data to obtain the final dataset

The merge strategy is based on the **image hash calculation**